# Challenge: Circuit Extraction

Our normal simplification procedure always yields ZX-diagrams that can be turned back into (ancilla-free) circuits.

**Challenge**: find a means of deterministically realising the ZX-diagram g2 below, _without increasing T-count_.

In [1]:
import random, math, os, time
import sys; sys.path.append('..')
import pyzx as zx
%config InlineBackend.figure_format = 'svg'

Here, we load the circuit and do our "normal" simplification, which reduces T-count from 21 to 15.

In [2]:
c = zx.Circuit.load('../circuits/Fast/tof_3_before').to_basic_gates()
g = c.to_graph()
g1 = g.copy()
zx.simplify.full_reduce(g1)
print("old T-count: %s --> new T-count: %s" % (zx.tcount(g), zx.tcount(g1)))

old T-count: 21 --> new T-count: 15


...and indeed the resulting ZX-diagram can be transformed back into a circuit.

In [3]:
c1 = zx.extract.streaming_extract(g1.copy())
zx.d3.draw(c1.to_graph())

This function will be removed. Please use zx.draw or zx.draw_d3 instead.


At this point, we can use the simplification rule **gadget_phasepoly** to reduce the T-count by _one more_, down to 14.

In [4]:
g2 = g1.copy()

while True:
    m = zx.rules.match_gadgets_phasepoly(g2)
    if len(m) != 0: zx.rules.apply_gadget_phasepoly(g2, m)
    else: break
print("old T-count: %s --> new T-count: %s" % (zx.tcount(g1), zx.tcount(g2)))

old T-count: 15 --> new T-count: 14


...but the circuit no longer extracts!

In [5]:
try:
    c2 = zx.extract.streaming_extract(g2.copy())
except ValueError as e:
    print("***Error in extraction***: %s" % e)

***Error in extraction***: Did not find any completely reducable row


Here is the ZX-diagram. **Can you extract it?**

In [7]:
zx.draw(g2)